## 6.01 - Supervised Learning Model Comparison

Recall the "data science process."

1. Define the problem.
2. Gather the data.
3. Explore the data.
4. Model the data.
5. Evaluate the model.
6. Answer the problem.

In this lab, we're going to focus mostly on creating (and then comparing) many regression and classification models. Thus, we'll define the problem and gather the data for you.

Most of the questions requiring a written response can be written in 2-3 sentences.

---

### Step 1: Define the problem.

You are a data scientist with a financial services company. Specifically, you want to leverage data in order to identify potential customers.

If you are unfamiliar with "401(k)s" or "IRAs," these are **two types** of retirement accounts. Very broadly speaking:
- You can put money for retirement into both of these accounts.
- The money in these accounts gets invested and hopefully has a lot more money in it when you retire.
- These are a little different from regular bank accounts in that there are certain tax benefits to these accounts. Also, employers frequently match money that you put into a 401k.
- If you want to learn more about them, check out [this site](https://www.nerdwallet.com/article/ira-vs-401k-retirement-accounts).

We will tackle one regression problem and one classification problem today.
- Regression: What features best predict one's income?
- Classification: Predict whether or not one is eligible for a 401k. 

Check out the data dictionary [here](http://fmwww.bc.edu/ec-p/data/wooldridge2k/401KSUBS.DES).

### NOTE: When predicting `inc`, you should pretend as though you do not have access to the `e401k`, the `p401k` variable, and the `pira` variable. When predicting `e401k`, you may use the entire dataframe if you wish.

## [Data Dictionary](http://fmwww.bc.edu/ec-p/data/wooldridge2k/401KSUBS.DES)

### Step 2: Gather the data.

##### 1. Read in the data from the repository.

In [101]:
import pandas as pd

In [102]:
df = pd.read_csv("401ksubs.csv")

In [103]:
df.head(3)

,e401k,inc,marr,male,age,fsize,nettfa,p401k,pira,incsq,agesq
0,0,13.170,0,0,40,1,4.575,0,1,173.4489,1600
1,1,61.230,0,1,35,1,154.000,1,0,3749.1130,1225
2,0,12.858,1,0,44,2,0.000,0,0,165.3282,1936


In [104]:
df.shape

(9275, 11)

##### 2. What are 2-3 other variables that, if available, would be helpful to have?

1. Education Level

2. Credit Score

3. Car/House owner?

##### 3. Suppose a peer recommended putting `race` into your model in order to better predict who to target when advertising IRAs and 401(k)s. Why would this be an unethical decision?

Yes. It is 2022, most things are offensive.

## Step 3: Explore the data.

##### 4. When attempting to predict income, which feature(s) would we reasonably not use? Why?

In [105]:
df.columns

Index(['e401k', 'inc', 'marr', 'male', 'age', 'fsize', 'nettfa', 'p401k',
       'pira', 'incsq', 'agesq'],
      dtype='object')

In [106]:
inc_df = df.drop(['e401k','pira','p401k'], axis=1)
inc_df.columns

Index(['inc', 'marr', 'male', 'age', 'fsize', 'nettfa', 'incsq', 'agesq'], dtype='object')

1. 'incsq' is not necessary. The presence of this variable will be perfectly correlated with the 'inc' target variable

##### 5. What two variables have already been created for us through feature engineering? Come up with a hypothesis as to why subject-matter experts may have done this.
> This need not be a "statistical hypothesis." Just brainstorm why SMEs might have done this!

incsq and agesq have been feature engineered for us.

There might be a quadratic relationship.

##### 6. Looking at the data dictionary, one variable description appears to be an error. What is this error, and what do you think the correct value would be?

inc and age column descriptions are incorrect. Should not be squared.

## Step 4: Model the data. (Part 1: Regression Problem)

Recall:
- Problem: What features best predict one's income?
- When predicting `inc`, you should pretend as though you do not have access to the `e401k`, the `p401k` variable, and the `pira` variable.

##### 7. List all modeling tactics we've learned that could be used to solve a regression problem (as of Wednesday afternoon of Week 6). For each tactic, identify whether it is or is not appropriate for solving this specific regression problem and explain why or why not.

In [107]:
inc_df.head(3)

,inc,marr,male,age,fsize,nettfa,incsq,agesq
0,13.170,0,0,40,1,4.575,173.4489,1600
1,61.230,0,1,35,1,154.000,3749.1130,1225
2,12.858,1,0,44,2,0.000,165.3282,1936


1. Linear Regression
2. K-Nearest Neighbours
3. Decision Trees
4. Random Forest
5. SVM

##### 8. Regardless of your answer to number 7, fit at least one of each of the following models to attempt to solve the regression problem above:
    - a multiple linear regression model
    - a k-nearest neighbors model
    - a decision tree
    - a set of bagged decision trees
    - a random forest
    - an Adaboost model
    - a support vector regressor
    
> As always, be sure to do a train/test split! In order to compare modeling techniques, you should use the same train-test split on each. I recommend setting a random seed here.

> You may find it helpful to set up a pipeline to try each modeling technique, but you are not required to do so!

## Import Regressors

In [108]:
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import BaggingRegressor, RandomForestRegressor, AdaBoostRegressor
from sklearn.svm import SVR

from sklearn.model_selection import train_test_split

In [109]:
from sklearn.preprocessing import StandardScaler, RobustScaler

In [110]:
inc_df_standard_scaled = pd.DataFrame(StandardScaler().fit_transform(inc_df),
                        columns = inc_df.columns)

inc_df_robust_scaled = pd.DataFrame(StandardScaler().fit_transform(inc_df),
                        columns = inc_df.columns)

In [111]:
inc_df_standard_scaled.head()

,inc,marr,male,age,fsize,nettfa,incsq,agesq
0,-1.082858,-1.300887,-0.506898,-0.104886,-1.235500,-0.226651,-0.648965,-0.216227
1,0.912268,-1.300887,1.972784,-0.590372,-1.235500,2.109561,0.542404,-0.634940
2,-1.095810,0.768706,-0.506898,0.283503,-0.580086,-0.298179,-0.651671,0.158941
3,2.475242,0.768706,1.972784,0.283503,-0.580086,0.042656,2.550909,0.158941
4,-0.690807,-1.300887,-0.506898,1.157377,-1.235500,-0.009720,-0.536366,1.133706


In [112]:
inc_df_robust_scaled.head()

,inc,marr,male,age,fsize,nettfa,incsq,agesq
0,-1.082858,-1.300887,-0.506898,-0.104886,-1.235500,-0.226651,-0.648965,-0.216227
1,0.912268,-1.300887,1.972784,-0.590372,-1.235500,2.109561,0.542404,-0.634940
2,-1.095810,0.768706,-0.506898,0.283503,-0.580086,-0.298179,-0.651671,0.158941
3,2.475242,0.768706,1.972784,0.283503,-0.580086,0.042656,2.550909,0.158941
4,-0.690807,-1.300887,-0.506898,1.157377,-1.235500,-0.009720,-0.536366,1.133706


In [113]:
X_train, X_test, y_train, y_test = train_test_split(inc_df_standard_scaled.drop(columns = ['inc', 'incsq']),
                                                    inc_df_standard_scaled['inc'],
                                                    test_size = .2,
                                                    random_state = 42)

In [114]:
X_train2, X_test2, y_train2, y_test2 = train_test_split(inc_df_robust_scaled.drop(columns = ['inc', 'incsq']),
                                                    inc_df_robust_scaled['inc'],
                                                    test_size = .2,
                                                    random_state = 42)

In [115]:
import numpy as np
np.random.seed(42)

In [116]:
linear_reg = LinearRegression()
linear_reg.fit(X_train, y_train)

knn_reg = KNeighborsRegressor()
knn_reg.fit(X_train, y_train)

cart_reg = DecisionTreeRegressor()
cart_reg.fit(X_train, y_train)

bagged_reg = BaggingRegressor()
bagged_reg.fit(X_train, y_train)

random_forest_reg = RandomForestRegressor()
random_forest_reg.fit(X_train, y_train)

adaboost_reg = AdaBoostRegressor()
adaboost_reg.fit(X_train, y_train)

support_vector_reg = SVR()
support_vector_reg.fit(X_train, y_train)

SVR()

In [117]:
linear_reg = LinearRegression()
linear_reg.fit(X_train2, y_train2)

knn_reg = KNeighborsRegressor()
knn_reg.fit(X_train2, y_train2)

cart_reg = DecisionTreeRegressor()
cart_reg.fit(X_train2, y_train2)

bagged_reg = BaggingRegressor()
bagged_reg.fit(X_train2, y_train2)

random_forest_reg = RandomForestRegressor()
random_forest_reg.fit(X_train2, y_train2)

adaboost_reg = AdaBoostRegressor()
adaboost_reg.fit(X_train2, y_train2)

support_vector_reg = SVR()
support_vector_reg.fit(X_train2, y_train2)

SVR()

##### 9. What is bootstrapping?

##### 10. What is the difference between a decision tree and a set of bagged decision trees? Be specific and precise!

##### 11. What is the difference between a set of bagged decision trees and a random forest? Be specific and precise!

##### 12. Why might a random forest be superior to a set of bagged decision trees?
> Hint: Consider the bias-variance tradeoff.

## Step 5: Evaluate the model. (Part 1: Regression Problem)

##### 13. Using RMSE, evaluate each of the models you fit on both the training and testing data.

In [118]:
from sklearn.metrics import mean_squared_error

In [119]:
for i in [linear_reg,knn_reg,cart_reg,bagged_reg,random_forest_reg,adaboost_reg,support_vector_reg]:

    mse_train = mean_squared_error(y_true = y_train,
                                    y_pred = i.predict(X_train))

    mse_test = mean_squared_error(y_true = y_test,
                                    y_pred = i.predict(X_test))

    print(f"{str(i)} train score={mse_train} and test score = {mse_test}")

LinearRegression() train score=0.7007080045472344 and test score = 0.7525898409972971
KNeighborsRegressor() train score=0.47035975853449735 and test score = 0.7020632620100997
DecisionTreeRegressor() train score=0.008831902189337711 and test score = 1.2602735016577913
BaggingRegressor() train score=0.13445641389765153 and test score = 0.7753880542456079
RandomForestRegressor() train score=0.10271242726665193 and test score = 0.7091909258545687
AdaBoostRegressor() train score=0.8113151320590656 and test score = 0.8955560764454514
SVR() train score=0.6176208276513294 and test score = 0.6734706247312124


##### 14. Based on training RMSE and testing RMSE, is there evidence of overfitting in any of your models? Which ones?

##### 15. Based on everything we've covered so far, if you had to pick just one model as your final model to use to answer the problem in front of you, which one model would you pick? Defend your choice.

##### 16. Suppose you wanted to improve the performance of your final model. Brainstorm 2-3 things that, if you had more time, you would attempt.

## Step 4: Model the data. (Part 2: Classification Problem)

Recall:
- Problem: Predict whether or not one is eligible for a 401k.
- When predicting `e401k`, you may use the entire dataframe if you wish.

##### 17. While you're allowed to use every variable in your dataframe, mention at least one disadvantage of using `p401k` in your model.

In [120]:
df.head()

,e401k,inc,marr,male,age,fsize,nettfa,p401k,pira,incsq,agesq
0,0,13.170,0,0,40,1,4.575,0,1,173.4489,1600
1,1,61.230,0,1,35,1,154.000,1,0,3749.1130,1225
2,0,12.858,1,0,44,2,0.000,0,0,165.3282,1936
3,0,98.880,1,1,44,2,21.800,0,0,9777.2540,1936
4,0,22.614,0,0,53,1,18.450,0,0,511.3930,2809


## Import Classifiers

In [121]:
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import BaggingClassifier, RandomForestClassifier, AdaBoostClassifier
from sklearn.svm import SVC

In [123]:
df.head()

,e401k,inc,marr,male,age,fsize,nettfa,p401k,pira,incsq,agesq
0,0,13.170,0,0,40,1,4.575,0,1,173.4489,1600
1,1,61.230,0,1,35,1,154.000,1,0,3749.1130,1225
2,0,12.858,1,0,44,2,0.000,0,0,165.3282,1936
3,0,98.880,1,1,44,2,21.800,0,0,9777.2540,1936
4,0,22.614,0,0,53,1,18.450,0,0,511.3930,2809


In [124]:
inc_df_standard_scaled = pd.DataFrame(StandardScaler().fit_transform(df),
                        columns = df.columns)

In [125]:
inc_df_standard_scaled.head()

,e401k,inc,marr,male,age,fsize,nettfa,p401k,pira,incsq,agesq
0,-0.803173,-1.082858,-1.300887,-0.506898,-0.104886,-1.235500,-0.226651,-0.617776,1.712236,-0.648965,-0.216227
1,1.245062,0.912268,-1.300887,1.972784,-0.590372,-1.235500,2.109561,1.618709,-0.584032,0.542404,-0.634940
2,-0.803173,-1.095810,0.768706,-0.506898,0.283503,-0.580086,-0.298179,-0.617776,-0.584032,-0.651671,0.158941
3,-0.803173,2.475242,0.768706,1.972784,0.283503,-0.580086,0.042656,-0.617776,-0.584032,2.550909,0.158941
4,-0.803173,-0.690807,-1.300887,-0.506898,1.157377,-1.235500,-0.009720,-0.617776,-0.584032,-0.536366,1.133706


In [126]:
X_train, X_test, y_train, y_test = train_test_split(inc_df_standard_scaled.drop(columns = ['e401k', 'p401k']),
                                                    [1 if inc_df_standard_scaled['e401k'][i] > 0 else 0 for i in range(inc_df_standard_scaled.shape[0])],
                                                    ## I ran the above line because when I scaled e401k, e401k was no longer binary!
                                                    ## I needed to turn my Y vector into a discrete variable.
                                                    test_size = .2,
                                                    random_state = 42)

In [127]:
np.random.seed(42)

In [128]:
logreg_class = LogisticRegression()
logreg_class.fit(X_train, y_train)

knn_class = KNeighborsClassifier()
knn_class.fit(X_train, y_train)

cart_class = DecisionTreeClassifier()
cart_class.fit(X_train, y_train)

bagged_class = BaggingClassifier()
bagged_class.fit(X_train, y_train)

random_forest_class = RandomForestClassifier()
random_forest_class.fit(X_train, y_train)

adaboost_class = AdaBoostClassifier()
adaboost_class.fit(X_train, y_train)

support_vector_class = SVC()
support_vector_class.fit(X_train, y_train)

SVC()

##### 18. List all modeling tactics we've learned that could be used to solve a classification problem (as of Wednesday afternoon of Week 6). For each tactic, identify whether it is or is not appropriate for solving this specific classification problem and explain why or why not.

##### 19. Regardless of your answer to number 18, fit at least one of each of the following models to attempt to solve the classification problem above:
    - a logistic regression model
    - a k-nearest neighbors model
    - a decision tree
    - a set of bagged decision trees
    - a random forest
    - an Adaboost model
    - a support vector classifier
    
> As always, be sure to do a train/test split! In order to compare modeling techniques, you should use the same train-test split on each. I recommend using a random seed here.

> You may find it helpful to set up a pipeline to try each modeling technique, but you are not required to do so!

## Step 5: Evaluate the model. (Part 2: Classfication Problem)

##### 20. Suppose our "positive" class is that someone is eligible for a 401(k). What are our false positives? What are our false negatives?

##### 21. In this specific case, would we rather minimize false positives or minimize false negatives? Defend your choice.

##### 22. Suppose we wanted to optimize for the answer you provided in problem 21. Which metric would we optimize in this case?

##### 23. Suppose that instead of optimizing for the metric in problem 21, we wanted to balance our false positives and false negatives using `f1-score`. Why might [f1-score](https://en.wikipedia.org/wiki/F1_score) be an appropriate metric to use here?

##### 24. Using f1-score, evaluate each of the models you fit on both the training and testing data.

In [129]:
from sklearn.metrics import f1_score

In [130]:
for i in [logreg_class,knn_class,cart_class,bagged_class,random_forest_class,adaboost_class,support_vector_class]:

    f1_train = f1_score(y_true = y_train,
                        y_pred = i.predict(X_train))
    f1_test = f1_score(y_true = y_test,
                        y_pred = i.predict(X_test))

    print(f"{str(i)} train F1 score={mse_train} and test F1 score = {mse_test}")

LogisticRegression() train F1 score=0.6176208276513294 and test F1 score = 0.6734706247312124
KNeighborsClassifier() train F1 score=0.6176208276513294 and test F1 score = 0.6734706247312124
DecisionTreeClassifier() train F1 score=0.6176208276513294 and test F1 score = 0.6734706247312124
BaggingClassifier() train F1 score=0.6176208276513294 and test F1 score = 0.6734706247312124
RandomForestClassifier() train F1 score=0.6176208276513294 and test F1 score = 0.6734706247312124
AdaBoostClassifier() train F1 score=0.6176208276513294 and test F1 score = 0.6734706247312124
SVC() train F1 score=0.6176208276513294 and test F1 score = 0.6734706247312124


##### 25. Based on training f1-score and testing f1-score, is there evidence of overfitting in any of your models? Which ones?

##### 26. Based on everything we've covered so far, if you had to pick just one model as your final model to use to answer the problem in front of you, which one model would you pick? Defend your choice.

##### 27. Suppose you wanted to improve the performance of your final model. Brainstorm 2-3 things that, if you had more time, you would attempt.

## Step 6: Answer the problem.

##### BONUS: Briefly summarize your answers to the regression and classification problems. Be sure to include any limitations or hesitations in your answer.

- Regression: What features best predict one's income?
- Classification: Predict whether or not one is eligible for a 401k.